# Setup

In [368]:
# # Init pyspark
# from pyspark import SparkContext
# sc = SparkContext.getOrCreate()
# # Init sparksql -- Only used to format the output nicely!
# from pyspark.sql import SQLContext
# sqlContext = SQLContext(sc)

# rows = sc.textFile("/air_transit_2007.csv")
# #rows = sc.textFile("/Users/abel/Downloads/spare_time/working/a13544_600_spark/air_transit_2007.csv")

# data = rows.map(lambda line: line.split(","))
# # data.cache()

# import findspark
# findspark.init()
# A simple demo for working with SparkSQL and Tweets
from pyspark import SparkContext, SparkConf
from pyspark.sql import HiveContext, Row
from pyspark.sql.types import IntegerType
import json
import sys


inputFile = 'air_transit_2007.csv'
inputFile = '/Users/abel/Downloads/spare_time/working/a13544_600_spark/air_transit_2007.csv'

conf = SparkConf().setAppName("SparkSQLAirTransit")
SparkConf().set("spark.sql.legacy.timeParserPolicy","LEGACY")

sc = SparkContext.getOrCreate(conf=conf)
hiveCtx = HiveContext(sc)
print("Loading tweets from " + inputFile)


Loading tweets from /Users/abel/Downloads/spare_time/working/a13544_600_spark/air_transit_2007.csv


In [369]:
input = hiveCtx.read.option("header",True).csv(inputFile,inferSchema =True)
input.printSchema()



root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: integer (nullable = true)
 |-- TaxiOut: integer (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- C

In [370]:
# Sample Query

In [371]:

input.registerTempTable("air_transit")

myair_transits = hiveCtx.sql("SELECT * FROM air_transit  LIMIT 3")
print('myair_transits:' )
for item in myair_transits.collect():
    print(item, '\n')

myair_transits:
Row(Year=2007, Month=1, DayofMonth=1, DayOfWeek=1, DepTime='1232', CRSDepTime=1225, ArrTime='1341', CRSArrTime=1340, UniqueCarrier='WN', FlightNum=2891, TailNum='N351', ActualElapsedTime='69', CRSElapsedTime='75', AirTime='54', ArrDelay='1', DepDelay='7', Origin='SMF', Dest='ONT', Distance=389, TaxiIn=4, TaxiOut=11, Cancelled=0, CancellationCode=None, Diverted=0, CarrierDelay=0, WeatherDelay=0, NASDelay=0, SecurityDelay=0, LateAircraftDelay=0) 

Row(Year=2007, Month=1, DayofMonth=1, DayOfWeek=1, DepTime='1918', CRSDepTime=1905, ArrTime='2043', CRSArrTime=2035, UniqueCarrier='WN', FlightNum=462, TailNum='N370', ActualElapsedTime='85', CRSElapsedTime='90', AirTime='74', ArrDelay='8', DepDelay='13', Origin='SMF', Dest='PDX', Distance=479, TaxiIn=5, TaxiOut=6, Cancelled=0, CancellationCode=None, Diverted=0, CarrierDelay=0, WeatherDelay=0, NASDelay=0, SecurityDelay=0, LateAircraftDelay=0) 

Row(Year=2007, Month=1, DayofMonth=1, DayOfWeek=1, DepTime='2206', CRSDepTime=2130, A

## Q1
Compute the total number of records.

In [372]:
# Response...
df = input

# mycount = hiveCtx.sql("SELECT count(*) as mycount FROM air_transit")
# print('mycount=', mycount.collect()[0]['mycount'] )

print('total number of records.',df.count() )

total number of records. 7453215


In [373]:
## Q2
#Find total number of operated flights per month, sorted by the month..


# mymonth = hiveCtx.sql("select Month, count(*) as flight_number from air_transit group by Month order by Month LIMIT 100")
# print('total number of operated flights per month:')  
# for item in mymonth.collect():
#     print(item['Month'], 'month', item['flight_number'])
    
from pyspark.sql.functions import col

print('Q2 total number of operated flights per month, sorted by the month:')

df1 = df.groupBy("Month").count().sort('Month',ascending=True) 
for item in df1.rdd.collect():
    print(item['Month'], item['count'])



Q2 total number of operated flights per month, sorted by the month:
1 621559
2 565604
3 639209
4 614648
5 631609
6 629280
7 648560
8 653279
9 600187
10 629992
11 605149
12 614139


In [374]:
## Q3
# Find the plane with the highest number of flights. Each plane has a unique TailNum

print('Q3: highest number of flights. Each plane has a unique TailNum')
df1 = df.filter(~col('TailNum').isin(['0', '000000']))
df1 = df1.groupBy("TailNum").count().sort('count',ascending=False)

if df1 is not None:
    for item in df1.rdd.collect()[:3]:
        print(item['TailNum'], item['count'])
    print('highest_number_filghts: ', df1.collect()[0]['TailNum'] )


Q3: highest number of flights. Each plane has a unique TailNum
N655BR 4483
N479HA 4359
N651BR 4346
highest_number_filghts:  N655BR


In [375]:
## Q4
# Compute the total flight time of each airplane, sorted by flight time in descending order.
# total_flight_times = hiveCtx.sql("SELECT TailNum, SUM (AirTime) as total_fight_time FROM air_transit GROUP BY TailNum order by total_fight_time desc LIMIT 100")
# print('-'*10, 'We make ', '-'*10)
# print('total flight time of each airplane: ')
# for item in  total_flight_times.collect():
#     print(item['TailNum'], ' total fight time is', item['total_fight_time'])

print('Q4:total flight time of each airplane, sorted by flight time in descending order')
df1 = df.groupBy("TailNum").agg(F.sum(df['AirTime']).alias('result')).orderBy('result',ascending=False)
if df1 is not None:
    for item in df1.rdd.collect()[:3]:
        print(item['TailNum'], item['result'])

Q4:total flight time of each airplane, sorted by flight time in descending order
N556AS 532213.0
N557UA 259376.0
N597UA 254760.0


In [376]:
# q5 Find the busiest airport (in terms of number of departures + arrivals of all operated flights) for each month.
from pyspark.sql.functions import col
import operator
df = input

for month in range(1, 12+1):
    print(month, '-'*10)
    df1=df.filter(col('Month').isin([month])).groupBy('Origin').count()
    df2=df.filter(col('Month').isin([month])).groupBy('Dest').count()
    #print(type(df1), type(df2))
    k_v = {}
    for orgin in df1.collect():
#         print(orgin['Origin'],orgin['count'])
        k_v[orgin['Origin']] = orgin['count']
    for dest in df2.collect():
        if k_v.get(dest['Dest']) :
#             print(dest)
#             print('before',k_v[dest['Dest']])
            k_v[dest['Dest']] += dest['count']
#             print('after',k_v[dest['Dest']])

    print(k_v)
    if k_v is not None:
        month_max, thecount = max(k_v.items(), key=operator.itemgetter(1))
        print('month_max:', month_max, ' ,departures + arrivals of this month:', thecount)



1 ----------
{'BGM': 237, 'DLG': 26, 'PSE': 129, 'MSY': 5535, 'GEG': 2355, 'SNA': 8859, 'BUR': 5399, 'GRB': 1508, 'GTF': 495, 'IDA': 537, 'GRR': 2775, 'EUG': 1004, 'PSG': 124, 'PVD': 4296, 'GSO': 2800, 'MYR': 990, 'ISO': 17, 'OAK': 12600, 'MSN': 2348, 'COD': 186, 'BTM': 123, 'FAR': 999, 'FSM': 367, 'MQT': 170, 'SCC': 124, 'DCA': 15605, 'RFD': 126, 'MLU': 571, 'CID': 1944, 'LWS': 123, 'GTR': 163, 'WRG': 124, 'HLN': 308, 'LEX': 1749, 'RDM': 605, 'FLO': 111, 'ORF': 3325, 'SCE': 135, 'EVV': 933, 'KTN': 372, 'CRW': 587, 'CWA': 306, 'SAV': 2106, 'TRI': 154, 'CDV': 124, 'CMH': 6874, 'MOD': 526, 'CAK': 1428, 'TYR': 246, 'ADK': 16, 'CHO': 272, 'MOB': 1034, 'PNS': 1731, 'MCN': 184, 'CEC': 172, 'LIH': 2762, 'IAH': 36883, 'HNL': 11434, 'SHV': 1606, 'ERI': 252, 'SJC': 10143, 'CVG': 19159, 'TOL': 363, 'BUF': 4756, 'LGA': 21243, 'TLH': 622, 'ACT': 364, 'HPN': 1735, 'RDD': 303, 'AUS': 7547, 'MLI': 1552, 'SJU': 4348, 'ATW': 979, 'DHN': 284, 'AVL': 536, 'GJT': 907, 'LGB': 2291, 'LYH': 178, 'BFL': 937, '

{'BGM': 231, 'DLG': 26, 'PSE': 120, 'MSY': 5650, 'GEG': 2641, 'SNA': 8862, 'BUR': 5408, 'GRB': 1460, 'GTF': 576, 'IDA': 557, 'GRR': 2713, 'LWB': 2, 'EUG': 977, 'PSG': 120, 'PVD': 4230, 'GSO': 2429, 'MYR': 1163, 'OAK': 12299, 'MSN': 2302, 'COD': 179, 'BTM': 137, 'FAR': 922, 'FSM': 375, 'MQT': 163, 'SCC': 120, 'DCA': 15153, 'RFD': 110, 'MLU': 496, 'CID': 1780, 'LWS': 114, 'GTR': 172, 'MTH': 68, 'WRG': 120, 'HLN': 285, 'LEX': 1685, 'RDM': 586, 'FLO': 112, 'ORF': 3284, 'SCE': 110, 'EVV': 961, 'KTN': 360, 'CRW': 632, 'CWA': 288, 'SAV': 2359, 'TRI': 165, 'CDV': 120, 'CMH': 6521, 'MOD': 514, 'CAK': 1397, 'TYR': 240, 'ADK': 18, 'CHO': 276, 'MOB': 918, 'PNS': 1667, 'MCN': 181, 'CEC': 169, 'LIH': 2559, 'IAH': 34358, 'HNL': 11149, 'SHV': 1560, 'ERI': 232, 'SJC': 10054, 'CVG': 18552, 'TOL': 351, 'BUF': 4728, 'TLH': 671, 'LGA': 20674, 'ACT': 351, 'HPN': 2074, 'RDD': 300, 'AUS': 8031, 'MLI': 1493, 'SJU': 4282, 'ATW': 1027, 'DHN': 269, 'AVL': 504, 'GJT': 905, 'LGB': 2280, 'LYH': 172, 'BFL': 872, 'GFK

{'BGM': 124, 'INL': 116, 'DLG': 62, 'PSE': 186, 'MSY': 6533, 'GEG': 2934, 'SNA': 9091, 'BUR': 5730, 'GRB': 1408, 'GTF': 473, 'IDA': 591, 'GRR': 2961, 'LWB': 58, 'EUG': 1200, 'PSG': 124, 'PVD': 4350, 'GSO': 2625, 'MYR': 1310, 'OAK': 12969, 'MSN': 2221, 'COD': 309, 'FAR': 883, 'BTM': 170, 'FSM': 520, 'MQT': 231, 'SCC': 124, 'DCA': 15278, 'RFD': 115, 'MLU': 512, 'CID': 1977, 'LWS': 120, 'GTR': 169, 'WRG': 124, 'HLN': 352, 'LEX': 1760, 'RDM': 669, 'FLO': 108, 'ORF': 3388, 'SCE': 124, 'EVV': 1016, 'KTN': 496, 'CRW': 588, 'CWA': 290, 'SAV': 2510, 'TRI': 224, 'CDV': 124, 'CMH': 6495, 'MOD': 516, 'CAK': 1608, 'TYR': 247, 'ADK': 18, 'CHO': 170, 'MOB': 1114, 'PNS': 1862, 'MCN': 187, 'CEC': 184, 'LIH': 2784, 'IAH': 33148, 'HNL': 11912, 'SHV': 1437, 'ERI': 235, 'GST': 62, 'SJC': 10895, 'CVG': 19201, 'TOL': 368, 'BUF': 4591, 'TLH': 807, 'LGA': 20611, 'ACT': 418, 'HPN': 2391, 'RDD': 310, 'AUS': 9308, 'MLI': 1598, 'SJU': 4359, 'ATW': 1041, 'DHN': 290, 'AVL': 806, 'GJT': 832, 'LGB': 2494, 'LYH': 150, 

{'BGM': 124, 'INL': 62, 'PSE': 176, 'MSY': 6688, 'GEG': 2825, 'SNA': 9002, 'BUR': 5681, 'GRB': 1361, 'GTF': 426, 'IDA': 590, 'GRR': 2716, 'LWB': 62, 'EUG': 1112, 'PSG': 124, 'PVD': 4428, 'GSO': 2432, 'MYR': 1203, 'OAK': 12863, 'MSN': 2296, 'COD': 192, 'FAR': 760, 'BTM': 124, 'FSM': 546, 'MQT': 170, 'SCC': 124, 'DCA': 14959, 'RFD': 116, 'MLU': 516, 'CID': 2027, 'LWS': 124, 'GTR': 175, 'WRG': 124, 'HLN': 282, 'LEX': 1569, 'RDM': 596, 'FLO': 122, 'ORF': 3374, 'SCE': 124, 'EVV': 1034, 'KTN': 372, 'CRW': 575, 'CWA': 302, 'SAV': 2565, 'TRI': 237, 'CDV': 124, 'CMH': 6382, 'MOD': 510, 'CAK': 1826, 'TYR': 309, 'ADK': 16, 'CHO': 240, 'MOB': 1068, 'PNS': 1768, 'MCN': 132, 'CEC': 184, 'LIH': 2462, 'IAH': 33044, 'HNL': 10931, 'SHV': 1534, 'ERI': 240, 'SJC': 10728, 'CVG': 18356, 'TOL': 354, 'BUF': 4126, 'TLH': 778, 'LGA': 20975, 'ACT': 417, 'HPN': 2297, 'RDD': 304, 'AUS': 9430, 'MLI': 1758, 'SJU': 3122, 'ATW': 1132, 'DHN': 251, 'AVL': 742, 'GJT': 716, 'LGB': 2302, 'LYH': 114, 'BFL': 825, 'GFK': 225,

In [377]:
## Q6
# 6nd the airline with highest average delay of each type in March 2007. 
#Note: do not write separate code for each error type. 
# You should compute a single RDD where each row contains the delay type, 
# the airline that is worst regarding that delay type, and its average delay of that type in minutes.

import pyspark.sql.functions as F 
from pyspark.sql.functions import count, avg
from pyspark.sql.functions import  max as max_

# d_columns =  ['CarrierDelay','WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
print('1 all of group:')
df1 = df.groupBy("FlightNum") \
    .avg('CarrierDelay','WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay')
df1.show()

r = df1.groupBy("FlightNum").agg(F.sum(df1[1]+df1[2] + df1[3] + df1[4]).alias('result')).orderBy('result',ascending=False).head(1)
print('2 the filghtnum is:', r[0].FlightNum)

print('3 the max row:')
df1.filter(col('FlightNum').isin([r[0].FlightNum])).show()

1 all of group:
+---------+------------------+-------------------+------------------+--------------------+----------------------+
|FlightNum| avg(CarrierDelay)|  avg(WeatherDelay)|     avg(NASDelay)|  avg(SecurityDelay)|avg(LateAircraftDelay)|
+---------+------------------+-------------------+------------------+--------------------+----------------------+
|     1959|1.2679558011049723| 0.5041436464088398| 4.433701657458563|0.029005524861878452|    1.6505524861878453|
|     2142|3.2524964336661912| 0.9900142653352354|2.6390870185449358|                 0.0|       6.8830242510699|
|      496|2.0320776927604474|0.30341377280753384| 2.695703354914656|0.023543260741612712|     4.066509711595056|
|     1342|1.9002100840336134| 1.0777310924369747| 2.653361344537815|0.017857142857142856|     8.884453781512605|
|     2659| 4.656678082191781| 0.5136986301369864|            2.1875| 0.06763698630136987|     5.648972602739726|
|     1238| 4.135495078170237| 0.5964099594672843| 3.854082223508975|0.0

In [378]:
## q7
# Compute median, mean, and mode of columns 12-16, 19-21 and 25-29 
# for the flights in the third week of 2007. Exclude the non-numeric values.

from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType

print('filter out week 3')
mydf = df.filter(col('DayOfWeek').isin([3])) 

print('median of [12:16]:')
df1 = mydf.select(mydf.columns[12:16])
# df1.show()

# res  = df1.select(*[F.percentile_approx(c).alias(c) for c in df1.columns])
# res.show()
for name in ('CRSElapsedTime', 'AirTime', 'ArrDelay', 'DepDelay'):
    df1 = df1.withColumn(name, df1[name].cast(IntegerType()))
    quantiles = df1.approxQuantile("CRSElapsedTime", [0.25, 0.5, 0.75], 0)
    print(name, ' median: ',quantiles)

df2 = mydf.select(mydf.columns[19:21])
# df2.show()
res2  = df2.select(*[F.mean(c).alias(c) for c in df2.columns])
print('mean of [19:21] :')
res2.show()

df3 = mydf.select(mydf.columns[25:29])
# df3.show()
print('mode of [25:29] :')
# [df3.groupby(i).count().orderBy("count", ascending=False).first()[0] for i in df3.columns]
for i in df3.columns:
    t = df3.groupby(i).count().orderBy("count", ascending=False).first()
    if t is not None:
        print(t)

filter out week 3
median of [12:16]:
CRSElapsedTime  median:  [77.0, 108.0, 155.0]
AirTime  median:  [77.0, 108.0, 155.0]
ArrDelay  median:  [77.0, 108.0, 155.0]
DepDelay  median:  [77.0, 108.0, 155.0]
mean of [19:21] :
+-----------------+------------------+
|           TaxiIn|           TaxiOut|
+-----------------+------------------+
|6.592582320192348|16.593127845871546|
+-----------------+------------------+

mode of [25:29] :
Row(WeatherDelay=0, count=1069792)
Row(NASDelay=0, count=936107)
Row(SecurityDelay=0, count=1087801)
Row(LateAircraftDelay=0, count=970520)


In [384]:
## Q8
#Assume that a passenger wants to travel from Philadelphia International Airport (airport code: PHL) 
# to Los Angeles International Airport (airport code: LAX), and then go back to Philadelphia (PHL). 
# He departs PHL not earlier than 5:59 am (scheduled time), stays at least 3:01 hours in Los Angeles and then arrive at PHL not later than 11pm. Based on the "scheduled" times, find which carrier has the highest number of flights with these constraints. 
# Limit your analysis to February 2007 and use scheduled times.

from pyspark.sql.functions import col
import pyspark.sql.functions as F


df1 = df.filter(col('Month').isin([2]) & df.DepTime.isNotNull() & df.ArrTime.isNotNull() ) 
# filter datetime is too early data
df1 = df1.filter( df.DepTime > '500' ) 
# df1 = df1.where(col("DepTime").isNotNull()).collect()
# df1 = df1.where(col("ArrTime").isNotNull()).collect()

df1 = df1.filter(col('Origin').isin(['PHL', 'LAX'])).filter(col('Dest').isin(['PHL', 'LAX']))
# df1.show()

# df2 = df1.filter(col('DepTime').isin([2])) 
print('stay 3:0 hour, it means before we shoulf flight here at least before back-fight start time - 3hours')
dates = ("1970-01-01 5:59:00", "1970-01-01 20:00:00", "1970-01-01 23:00:00")
# date_from, date_to = [to_date(lit(s)).cast(TimestampType()) for s in dates]


df2 = df1.withColumn('DepTime_Date', F.to_timestamp('DepTime', 'hhmm'))
df3 = df2.withColumn('ArrTime_Date', F.to_timestamp('ArrTime', 'hhmm'))
# df2.show()

df4 = df3.where( ((col('DepTime_Date') > dates[0]) & (col('ArrTime_Date') < dates[1])) |
           (col('ArrTime_Date') < dates[2])
         )
# df4.show()
if df4 is not None:
    df5 = df4.groupBy('UniqueCarrier').count()
    k_v = {}
    for orgin in df5.rdd.collect():

        k_v[orgin['UniqueCarrier']] = orgin['count']
    print(k_v)
    if k_v is not None:
        carriar_max = max(k_v.items(), key=operator.itemgetter(1))[0]
        print('max UniqueCarrier :', carriar_max)

stay 3:0 hour, it means before we shoulf flight here at least before back-fight start time - 3hours
{'UA': 17, 'US': 20, 'WN': 27}
max UniqueCarrier : WN


In [388]:
## Q9
#Generate the `departure flights` board of the Los Angeles Airport at 12 Jan 2007 at 13:00. 
# The board should contain flights with actual departure times between 12:00 and 14:00, 
# sorted by scheduled departure time. The resulting table should at least contain scheduled departure time, 
# actual departure time (if departed), 
# airline code, and destination
df1 = df.filter(col('Month').isin([1])).filter(col('DayofMonth').isin([1]))
df1 = df1.filter( df.CRSDepTime > '500' ) 

dates = ("1970-01-01 12:00:00", "1970-01-01 14:00:00")
dates = ("1970-01-01 00:01:00", "1970-01-01 23:59:00")
# date_from, date_to = [to_date(lit(s)).cast(TimestampType()) for s in dates]

df1 = df1.withColumn(
    "CRSDepTime",
    F.when(F.col("CRSDepTime").isNull(), '0')
    .otherwise(F.col("CRSDepTime").cast("string"))
)
df2 = df1.withColumn('DepTime_Date', F.to_timestamp('CRSDepTime', 'hhmm'))

# df2.show()
# print('@'*20)

df4 = df2.where( 
    ((col('DepTime_Date') > dates[0]) & (col('DepTime_Date') < dates[1]))
         )

print('-'*10, 'departure flights(limit 100)', '-'*10)
for item in df4.rdd.collect()[:100]:
#     print(item)
    print(item['CRSArrTime'],item['DepTime'], item['FlightNum'], item['Dest'])

---------- departure flights(limit 100) ----------
1340 1232 2891 ONT
1410 1251 933 SAN
1205 1042 1554 SAN
1225 1045 1502 SEA
1300 1140 1596 SNA
1300 1233 851 LAS
1155 1035 1184 OAK
1315 1101 324 PHX
1450 1245 1684 PHX
1250 1136 2037 SJC
1220 1052 1977 SMF
1340 1215 2588 SMF
1225 959 1477 CLE
1225 1058 2108 DAL
1115 1014 1361 LIT
1330 1224 2584 MCI
1450 1132 338 MCO
1250 1151 2142 MDW
1155 1207 2968 MDW
1340 1159 2747 OKC
1445 1323 271 PHX
1405 1122 1167 ALB
1245 1007 2844 BDL
1115 1036 653 BNA
1405 1213 163 BWI
1325 1120 2503 CMH
1140 1046 430 FLL
1315 1213 2792 FLL
1220 1018 3026 IAD
1500 1303 1265 IND
1515 1242 2182 ISP
1145 1055 2374 JAX
1250 1121 250 MDW
1415 1125 845 MHT
1335 1253 710 PBI
1320 1055 2951 PHL
1250 1034 2999 PIT
1325 1039 880 PVD
1155 1058 188 DAL
1155 1015 2673 HOU
1325 1228 1075 MCI
1415 1231 1590 PHX
1120 1051 2970 LAX
1440 1024 379 MDW
1345 1142 2980 MSP
1230 1039 2462 STL
1357 1349 2622 EWR
1426 1241 2498 CLE
1125 1004 2815 EWR
1342 1036 2307 JAX
1344 1240 2461